In [41]:
# Bibliotecas Generales
from IPython.display import display, Markdown
import pandas as pd
import gc
import random
import os
import pickle
import tqdm
import logging
import itertools
import re

# K-Fold y Regresión
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error
import math
from statistics import mean, stdev
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import LeaveOneOut, KFold, cross_val_score

# Hyperopt
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK, space_eval

# XGBoost
import xgboost as xgb
import numpy as np
from scipy.stats import uniform
from sklearn.metrics import balanced_accuracy_score, make_scorer, confusion_matrix
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

# W2V (Word2Vec)
from nltk.corpus import stopwords
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import gensim
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler

# Configuración
random_state = 42


## Cargo los datos

In [14]:
# Load the competition data
comp_data = pd.read_csv("../competition_data.csv")

## Tuneo de datos

Retoque de fecha - puedo ahcerlo antes porque son pocas y oredecibles

In [15]:
comp_data['date'] = pd.to_datetime(comp_data['date'])
comp_data['month'] = comp_data['date'].dt.month
# comp_data['day'] = comp_data['date'].dt.day
comp_data['dayofweek'] = comp_data['date'].dt.dayofweek
# comp_data['hour'] = comp_data['date'].dt.hour

In [16]:
comp_data['is_pdp'].fillna(-1, inplace=True)
comp_data['is_pdp'] = comp_data['is_pdp'].replace({'True': 1, 'False': 0})
comp_data['is_pdp'] = comp_data['is_pdp'].astype(int)

In [17]:
comp_data["warranty"] = comp_data["warranty"].str.lower()
comp_data["warranty"] = comp_data["warranty"].str.replace("á", "a")
comp_data["warranty"] = comp_data["warranty"].str.replace("í", "i")

def has_warranty(value):
    if value == 'Sin garantia':
        return 0
    return 1

comp_data['has_warranty'] = comp_data['warranty'].apply(has_warranty)
comp_data["warranty_missing"] = (~comp_data["warranty"].isna()).astype(float)
comp_data["warranty_vendedor"] = comp_data["warranty"].str.contains("vendedor").astype(float)
comp_data["warranty_fabrica"] = comp_data["warranty"].str.contains("fabrica").astype(float)

In [18]:
add_tags = True

tags = []
names = []

if add_tags:
    if len(tags) == 0:
        #tags = comp_data["tags"].str.replace("[", "").str.replace("]", "").str.split(", ").apply(pd.Series).stack().value_counts()
        # more efficient
        tags = comp_data["tags"].str.replace("[", "").str.replace("]", "").str.split(", ").explode().value_counts()

    for tag in tags.index:
        comp_data["tag_" + tag] = comp_data["tags"].str.contains(tag).astype(int)

    comp_data["tags_count"] = comp_data["tags"].str.replace("[", "").str.replace("]", "").str.split(", ").apply(len)

    # comp_data = comp_data.drop("tags", axis=1)

polynomial features

In [19]:
comp_data['pdp_price'] = comp_data['is_pdp'] * comp_data['price']

In [20]:
poly_attrs = ["print_position", "offset", "price", "health", "original_price"]

for x in poly_attrs:
    comp_data[x + "2"] = comp_data[x] ** 2

for (x, y) in itertools.combinations(poly_attrs, 2):
    comp_data[x + "2 + " + y + "2"] = comp_data[x] ** 2 + comp_data[y] ** 2

## W2Vec 
en title

In [21]:
w2v =False

In [ ]:
if w2v:
    # Descarga de stopwords para español si no están descargadas ya
    nltk.download('stopwords')
    nltk.download('punkt')

    STOP_WORDS_SP = set(stopwords.words('spanish'))

    def iterate_LN_corpus(path):
        """
        Genera un iterador para recorrer los archivos de texto en un directorio.

        Args:
            path (str): Ruta al directorio que contiene los archivos.

        Yields:
            str: Texto contenido en cada archivo.
        """
        articles = os.listdir(path)
        random.shuffle(articles)
        for art in articles:
            with open(path + art, encoding="utf-8") as f:
                raw_text = f.read()
            yield(raw_text)

    def tokenizer(raw_text):
        """
        Tokeniza y preprocesa un texto.

        Args:
            raw_text (str): Texto sin procesar.

        Returns:
            list: Lista de oraciones, donde cada oración es una lista de palabras.
        """
        sentences = sent_tokenize(raw_text)
        sentences = [word_tokenize(e) for e in sentences]
        sentences = [[e2 for e2 in e1 if re.compile("[A-Za-z]").search(e2[0])] for e1 in sentences]
        sentences = [[e2.lower() for e2 in e1] for e1 in sentences]
        print(sentences)
        return(sentences)

    def gen_sentences(path):
        """
        Genera una lista de oraciones a partir de archivos de texto en un directorio.

        Args:
            path (str): Ruta al directorio que contiene los archivos de texto.

        Returns:
            list: Lista de oraciones.
        """
        sentences = []
        n_arts = len(os.listdir(path))
        for i, art in tqdm.tqdm(enumerate(iterate_LN_corpus(path)), total=n_arts):
            sentences.extend(tokenizer(art))
        return(sentences)

    def average_vectors(title_tokens, model, stopwords=None):
        """
        Calcula el vector promedio de un conjunto de tokens utilizando un modelo Word2Vec.

        Args:
            title_tokens (list): Lista de tokens.
            model (gensim.models.Word2Vec): Modelo Word2Vec.
            stopwords (set, optional): Conjunto de palabras stopwords. Defaults to None.

        Returns:
            numpy.ndarray: Vector promedio.
        """
        title_tokens = [e2 for e1 in title_tokens for e2 in e1]
        title_tokens = [e for e in title_tokens if e in model.wv]
        if stopwords is not None:
            title_tokens = [e for e in title_tokens if e not in stopwords]
        if len(title_tokens) == 0:
            output = np.zeros(model.wv.vector_size)
        else:
            output = np.array([model.wv.get_vector(e) for e in title_tokens]).mean(0)
        return output

    def dummy_tokenizer(text_tokens):
        """
        Tokenizador dummy que simplemente devuelve los tokens de texto sin procesar.

        Args:
            text_tokens (list): Lista de tokens.

        Returns:
            list: Misma lista de tokens de entrada.
        """
        return text_tokens


    #~ Análisis con datos de; TP ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    # Carga de datos desde un archivo CSV
    # comp_data = pd.read_csv("competition_data.csv")
    comp_data["title_tokens"] = comp_data["title"].map(tokenizer)

    # Creación del modelo Word2Vec
    w2v_tp = gensim.models.Word2Vec(vector_size= 300,     # Tamaño del vector
                                    window=5,             # Tamaño de ventana
                                    min_count=10,         # Frecuencia mínima de palabra
                                    negative=20,          # Número de ejemplos negativos para muestreo negativo
                                    sample=0.001,         # Submuestreo de palabras frecuentes
                                    workers=8,            # Número de núcleos de CPU para entrenamiento paralelo
                                    sg=1)                 # Algoritmo: 1 para Skip-gram, 0 para CBOW

    # Creación del vocabulario a partir del corpus
    w2v_tp.build_vocab([e2 for e1 in comp_data["title_tokens"].values for e2 in e1],
                    progress_per=10000)

    # Entrenamiento del modelo Word2Vec
    w2v_tp.train([e2 for e1 in comp_data["title_tokens"].values for e2 in e1],
                total_examples=w2v_tp.corpus_count,
                epochs=30, report_delay=1)

    ## Ejercicio de predicción con word2vec

    # Obtención de embeddings de títulos utilizando el modelo Word2Vec
    title_embs = comp_data["title_tokens"].map(lambda x: average_vectors(x, w2v_tp, STOP_WORDS_SP))
    title_embs= np.array(title_embs.to_list())

In [ ]:
w2v =False
if w2v:
    # Descarga de stopwords para español si no están descargadas ya
    nltk.download('stopwords')
    nltk.download('punkt')

    STOP_WORDS_SP = set(stopwords.words('spanish'))

    def iterate_LN_corpus(path):
        """
        Genera un iterador para recorrer los archivos de texto en un directorio.

        Args:
            path (str): Ruta al directorio que contiene los archivos.

        Yields:
            str: Texto contenido en cada archivo.
        """
        articles = os.listdir(path)
        random.shuffle(articles)
        for art in articles:
            with open(path + art, encoding="utf-8") as f:
                raw_text = f.read()
            yield(raw_text)

    def tokenizer(raw_text):
        """
        Tokeniza y preprocesa un texto.

        Args:
            raw_text (str): Texto sin procesar.

        Returns:
            list: Lista de oraciones, donde cada oración es una lista de palabras.
        """
        sentences = sent_tokenize(raw_text)
        sentences = [word_tokenize(e) for e in sentences]
        sentences = [[e2 for e2 in e1 if re.compile("[A-Za-z]").search(e2[0])] for e1 in sentences]
        sentences = [[e2.lower() for e2 in e1] for e1 in sentences]
        print(sentences)
        return(sentences)

    def gen_sentences(path):
        """
        Genera una lista de oraciones a partir de archivos de texto en un directorio.

        Args:
            path (str): Ruta al directorio que contiene los archivos de texto.

        Returns:
            list: Lista de oraciones.
        """
        sentences = []
        n_arts = len(os.listdir(path))
        for i, art in tqdm.tqdm(enumerate(iterate_LN_corpus(path)), total=n_arts):
            sentences.extend(tokenizer(art))
        return(sentences)

    def average_vectors(title_tokens, model, stopwords=None):
        """
        Calcula el vector promedio de un conjunto de tokens utilizando un modelo Word2Vec.

        Args:
            title_tokens (list): Lista de tokens.
            model (gensim.models.Word2Vec): Modelo Word2Vec.
            stopwords (set, optional): Conjunto de palabras stopwords. Defaults to None.

        Returns:
            numpy.ndarray: Vector promedio.
        """
        title_tokens = [e2 for e1 in title_tokens for e2 in e1]
        title_tokens = [e for e in title_tokens if e in model.wv]
        if stopwords is not None:
            title_tokens = [e for e in title_tokens if e not in stopwords]
        if len(title_tokens) == 0:
            output = np.zeros(model.wv.vector_size)
        else:
            output = np.array([model.wv.get_vector(e) for e in title_tokens]).mean(0)
        return output

    def dummy_tokenizer(text_tokens):
        """
        Tokenizador dummy que simplemente devuelve los tokens de texto sin procesar.

        Args:
            text_tokens (list): Lista de tokens.

        Returns:
            list: Misma lista de tokens de entrada.
        """
        return text_tokens


    #~ Análisis con datos de; TP ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    # Carga de datos desde un archivo CSV
    # comp_data = pd.read_csv("competition_data.csv")
    comp_data["title_tokens"] = comp_data["title"].map(tokenizer)

    # Creación del modelo Word2Vec
    w2v_tp = gensim.models.Word2Vec(vector_size= 300,     # Tamaño del vector
                                    window=5,             # Tamaño de ventana
                                    min_count=10,         # Frecuencia mínima de palabra
                                    negative=20,          # Número de ejemplos negativos para muestreo negativo
                                    sample=0.001,         # Submuestreo de palabras frecuentes
                                    workers=8,            # Número de núcleos de CPU para entrenamiento paralelo
                                    sg=1)                 # Algoritmo: 1 para Skip-gram, 0 para CBOW

    # Creación del vocabulario a partir del corpus
    w2v_tp.build_vocab([e2 for e1 in comp_data["title_tokens"].values for e2 in e1],
                    progress_per=10000)

    # Entrenamiento del modelo Word2Vec
    w2v_tp.train([e2 for e1 in comp_data["title_tokens"].values for e2 in e1],
                total_examples=w2v_tp.corpus_count,
                epochs=30, report_delay=1)

    ## Ejercicio de predicción con word2vec

    # Obtención de embeddings de títulos utilizando el modelo Word2Vec
    title_embs = comp_data["title_tokens"].map(lambda x: average_vectors(x, w2v_tp, STOP_WORDS_SP))
    title_embs= np.array(title_embs.to_list())

## PCA 
sobre los embedding de w2v


In [22]:
if w2v:
    # Confirmamos que las matriz de covarianza de las variables estandarizadas es igual a la de correlación de las originales
    scaler = StandardScaler(with_std=True, with_mean=True)
    title_embs_scaled = pd.DataFrame(scaler.fit_transform(title_embs))
    title_embs_scaled.cov()

    n_components = 30
    # Realiza el análisis de componentes principales (PCA) en los datos escalados
    pca_title_embs = PCA(n_components = n_components)
    pca_title_embs.fit(title_embs_scaled)

    # Crea un DataFrame para almacenar los resultados del PCA
    scores = pd.DataFrame(pca_title_embs.transform(title_embs_scaled), index=comp_data.index)
    scores.head()

    # Veamos qué valores de los loadings se obtuvieron (cada fila es un vector de loadins)
    print(scores)


Une compdata con los embs de los titulos 

In [23]:
if w2v:
    # title_embs_df = pd.DataFrame(title_embs, columns=[f"emb_{i}" for i in range(title_embs.shape[1])])

    # # Concatenate comp_data with title_embs_df along the columns axis (axis=1)
    # comp_data = pd.concat([comp_data, title_embs_df], axis=1)
    comp_data = pd.concat([comp_data, scores], axis=1)


## Dropeo 
atributos que no ayudan al modelo

In [24]:
comp_data = comp_data.drop(
    columns=['accepts_mercadopago', 
            # 'available_quantity',
            # 'avg_gmv_item_domain_30days',
            # 'avg_gmv_item_sel',
            # 'avg_gmv_seller_bday',
            # 'avg_qty_orders_item_domain_30days',
            # 'avg_qty_orders_item_sel_30days',
            # 'avg_si_item_sel_30day',
            'benefit',              
            'boosted',
            'category_id',          
            # 'conversion',
            'date',             #chiche d efechas separar dia mes etc
            'deal_print_id',
            'domain_id',
            'etl_version',
            'free_shipping',    
            'fulfillment',
            'full_name',
            # 'health',
            # 'is_pdp',
            'product_id',
            'item_id',
            # 'listing_type_id', #hacer OHE
            # 'logistic_type',   #hacer OHE
            'main_picture',
            # 'offset',
            # 'original_price',
            # 'platform',        #hacer OHE
            # 'price',
            # 'print_position',
            'print_server_timestamp',
            # 'qty_items_dom',
            # 'qty_items_sel',
            'site_id',
            # 'sold_quantity',
            'tags',             #experimentar
            'title',            #W2vec
            # 'total_asp_item_domain_30days',
            # 'total_asp_item_sel_30days',
            # 'total_gmv_domain_bday',
            # 'total_gmv_item_30days',
            # 'total_items_domain',
            # 'total_items_seller',
            # 'total_orders_domain_30days',
            # 'total_orders_item_30days',
            # 'total_orders_sel_30days',
            # 'total_si_domain_30days',
            # 'total_si_item_30days',
            # 'total_si_sel_30days',
            # 'total_visits_domain',
            # 'total_visits_item',
            # 'total_visits_seller',
            'uid',
            'user_id',
            'warranty',
            # 'ROW_ID',
            # 'title_tokens',
            ]
)

In [25]:
comp_data.dtypes

available_quantity                     int64
avg_gmv_item_domain_30days           float64
avg_gmv_item_sel                     float64
avg_gmv_seller_bday                  float64
avg_qty_orders_item_domain_30days    float64
                                      ...   
offset2 + health2                    float64
offset2 + original_price2              int64
price2 + health2                     float64
price2 + original_price2               int64
health2 + original_price2            float64
Length: 74, dtype: object

## Validation Set - HOLDOUT SET

In [26]:

# Split into training and evaluation samples
train_data = comp_data[comp_data["ROW_ID"].isna()]
eval_data = comp_data[comp_data["ROW_ID"].notna()]
del comp_data
gc.collect()

# train_data = pd.get_dummies(train_data, columns=['is_pdp','listing_type_id','logistic_type','platform','month','dayofweek'])
y_train_all = train_data["conversion"]
X_train_all = train_data.drop(columns=["conversion", "ROW_ID"])
del train_data
gc.collect()

#hold out set
X_train, X_val, y_train, y_val = train_test_split(X_train_all, y_train_all, test_size=0.2, random_state=random_state)



## OHE 
en las categoricas

In [27]:

X_train = pd.get_dummies(X_train, columns=['is_pdp','listing_type_id','logistic_type','platform','month','dayofweek']) 

X_val = pd.get_dummies(X_val, columns=['is_pdp','listing_type_id','logistic_type','platform','month','dayofweek']) 

eval_data = pd.get_dummies(eval_data, columns=['is_pdp','listing_type_id','logistic_type','platform','month','dayofweek'])

X_train_all = pd.get_dummies(X_train_all, columns=['is_pdp','listing_type_id','logistic_type','platform','month','dayofweek'])



In [28]:

# align X_train and eval_data
eval_data_rowid = eval_data["ROW_ID"]

X_train_all, eval_data = X_train_all.align(eval_data, join='left', axis=1, fill_value=0)

# Concatenar los DataFrames
eval_data = pd.concat([eval_data, eval_data_rowid], axis=1)

# Ahora eval_data contiene las columnas de eval_data, eval_data_conversion y eval_data_rowid concatenadas


In [29]:
# X_train_all.dtypes
unique_data_types = X_train_all.dtypes.unique()
print(unique_data_types)

[dtype('int64') dtype('float64') dtype('int32') dtype('bool')]


In [30]:
eval_data.shape

(19211, 92)

## Modelo de aprendizaje - XGBoost

Hyperopt para buscar los parametros para xgb

In [51]:
greedy = False

if not greedy:    
# Define el espacio de búsqueda de hiperparámetros para XGBoost
    space = {
        'max_depth': hp.quniform('max_depth', 3, 11, 1),
        'learning_rate': hp.uniform('learning_rate', 0.01, 0.1),
        'gamma': hp.uniform('gamma', 1, 20),
        'reg_lambda': hp.uniform('reg_lambda', 0, 15),
        'subsample': hp.uniform('subsample', 0.5, 1),
        'min_child_weight': hp.uniform('min_child_weight', 0, 100),
        'colsample_bytree': hp.uniform('colsample_bytree', 0.65, 1),
        'n_estimators': hp.quniform('n_estimators', 500, 2500, 1)
    }

    # Función objetivo para la optimización
    def objective(params):
        params['n_estimators'] = int(params['n_estimators'])  # Redondea n_estimators a un valor entero
        params['max_depth'] = int(params['max_depth'])  # Redondea max_depth a un valor entero
        clf_xgb = xgb.XGBClassifier(objective='binary:logistic', seed=42, eval_metric='auc', tree_method='gpu_hist', **params)
        cv_scores = cross_val_score(clf_xgb, X_train, y_train, cv=KFold(4))
        mean_auc_roc = np.mean(cv_scores)
        return {'loss': 1 - mean_auc_roc, 'status': STATUS_OK}

    # Realiza la optimización con Hyperopt
    trials = Trials()
    best = fmin(objective, space, algo=tpe.suggest, max_evals=80, rstate=np.random.default_rng(22))

    # Obtiene los mejores hiperparámetros encontrados
    best_grid = space_eval(space, best)

    print("Mejores hiperparámetros encontrados para XGBoost:")
    print(best_grid)



  0%|          | 0/80 [00:00<?, ?trial/s, best loss=?]

 15%|█▌        | 12/80 [11:16<49:23, 43.58s/trial, best loss: 0.08910986944014165]  

In [ ]:
if not greedy:    
# Define el espacio de búsqueda de hiperparámetros para XGBoost
    space = {
        'max_depth': hp.quniform('max_depth', 3, 11, 1),
        'learning_rate': hp.uniform('learning_rate', 0.01, 0.1),
        'gamma': hp.uniform('gamma', 1, 20),
        'reg_lambda': hp.uniform('reg_lambda', 0, 15),
        'subsample': hp.uniform('subsample', 0.5, 1),
        'min_child_weight': hp.uniform('min_child_weight', 0, 100),
        'colsample_bytree': hp.uniform('colsample_bytree', 0.65, 1),
        'n_estimators': hp.quniform('n_estimators', 500, 2500, 1)
    }

    # Función objetivo para la optimización
    def objective(params):
        params['n_estimators'] = int(params['n_estimators'])  # Redondea n_estimators a un valor entero
        params['max_depth'] = int(params['max_depth'])  # Redondea max_depth a un valor entero
        clf_xgb = xgb.XGBClassifier(objective='binary:logistic', seed=42, eval_metric='auc', tree_method='gpu_hist', **params)
        cv_scores = cross_val_score(clf_xgb, X_train, y_train, cv=KFold(4))
        mean_auc_roc = np.mean(cv_scores)
        return {'loss': 1 - mean_auc_roc, 'status': STATUS_OK}

    # Realiza la optimización con Hyperopt
    trials = Trials()
    best = fmin(objective, space, algo=tpe.suggest, max_evals=80, rstate=np.random.default_rng(22))

    # Obtiene los mejores hiperparámetros encontrados
    best_grid = space_eval(space, best)

    print("Mejores hiperparámetros encontrados para XGBoost:")
    print(best_grid)



greedy serch apra buscar parametros de xgb

In [32]:
if greedy:
        
    #ver uan forma de tener mejor parametos que gerar
    params = {'max_depth': list(range(3, 12)),
            'learning_rate': uniform(scale = 0.05),
            'gamma': uniform(scale=10),               #chiche va de 1 a 20
            'reg_lambda': uniform(scale = 15),        # Parámetro de regularización.
            'subsample': uniform(0.5, 0.5),          # Entre 0.5 y 1.
            'min_child_weight': uniform(scale = 60),   #0 a 100
            'colsample_bytree': uniform(0.65, 0.35), # Entre 0.75 y 1.
            'n_estimators': list(range(700, 2500))    #ma que 500 tambein a la noche para que corra
            }

    l_auc = []
    l_grid = []
    start = time.time()
    random_state = 42
    best_score = 0
    best_estimator = None
    iterations = 80
    i=1
    for g in ParameterSampler(params, n_iter = iterations, random_state = random_state):
        clf_xgb = xgb.XGBClassifier(objective = 'binary:logistic', seed = random_state, eval_metric = 'auc', **g, tree_method = 'gpu_hist') #enable_categorical = True
        clf_xgb.fit(X_train, y_train, eval_set = [(X_val, y_val)], verbose = False)

        y_pred = clf_xgb.predict_proba(X_val)[:, 1] # Obtenemos la probabilidad de una de las clases (cualquiera).
        auc_roc = sklearn.metrics.roc_auc_score(y_val, y_pred)
        
        l_auc.append(auc_roc)
        l_grid.append(g)
        print(f'{i}-Roc-AUC actual; {auc_roc}')
        print(f'{i}-Con grid: {g}')
        i+=1
        # Guardamos si es mejor.
        if auc_roc > best_score:
            print('-'*10)
            print(f'Mejor valor de ROC-AUC encontrado: {auc_roc}')
            print(f'Grid{g}')
            print('-'*10)
            best_score = auc_roc
            best_grid = g
            best_estimator = clf_xgb

    end = time.time()

    df_aux = pd.DataFrame({'auc': l_auc, 'grid': l_grid})
    df_aux.to_csv('grid_search.csv', index = False)


Roc-AUC actual; 0.8851612430252037
Con grid: {'colsample_bytree': 0.7810890415965769, 'gamma': 9.50714306409916, 'learning_rate': 0.03659969709057025, 'max_depth': 7, 'min_child_weight': 9.361118426546192, 'n_estimators': 1166, 'reg_lambda': 1.4996237372700434, 'subsample': 0.7296244459829335}
----------
Mejor valor de ROC-AUC encontrado: 0.8851612430252037
Grid{'colsample_bytree': 0.7810890415965769, 'gamma': 9.50714306409916, 'learning_rate': 0.03659969709057025, 'max_depth': 7, 'min_child_weight': 9.361118426546192, 'n_estimators': 1166, 'reg_lambda': 1.4996237372700434, 'subsample': 0.7296244459829335}
----------
Roc-AUC actual; 0.8918087478020738
Con grid: {'colsample_bytree': 0.7667980138986576, 'gamma': 1.4286681792194078, 'learning_rate': 0.032544423647442644, 'max_depth': 7, 'min_child_weight': 58.19459112971966, 'n_estimators': 2215, 'reg_lambda': 14.078290635236252, 'subsample': 0.5003893829205072}
----------
Mejor valor de ROC-AUC encontrado: 0.8918087478020738
Grid{'colsam

me quedo con el mejor auc

In [33]:
if greedy: 
    print('ROC-AUC: %0.5f' % best_score)
    print('Grilla:', best_grid)
    print(f'Tiempo transcurrido: {str(end - start)} segundos')
    print(f'Tiempo de entrenamiento por iteración: {str(round((end - start) / iterations, 2))} segundos')

ROC-AUC: 0.89323
Grilla: {'colsample_bytree': 0.9778268531494976, 'gamma': 0.39186326675646255, 'learning_rate': 0.02089730158577894, 'max_depth': 11, 'min_child_weight': 32.87831299488524, 'n_estimators': 1156, 'reg_lambda': 8.527804256249375, 'subsample': 0.7879622876290423}
Tiempo transcurrido: 3318.050509929657 segundos
Tiempo de entrenamiento por iteración: 41.48 segundos


ahora con toda la data el prosible auc:

In [34]:
# y_pred = best_estimator.predict_proba(X_val)[:, 1]
# auc_roc = sklearn.metrics.roc_auc_score(y_val, y_pred)
# print('AUC-ROC test: %0.5f' % auc_roc)

AUC-ROC test: 0.89323


entreno con toda la data train+val

In [35]:
# best_grid = {'colsample_bytree': 0.7860370513913187, 'gamma': 4.070235476608438, 'learning_rate': 0.02486212527455788, 'max_depth': 11, 'min_child_weight': 14.046725484369038, 'n_estimators': 879, 'reg_lambda': 4.1399877303381505, 'subsample': 0.6481367528520412}

In [36]:
clf_xgb = xgb.XGBClassifier(objective = 'binary:logistic', seed = random_state, eval_metric = 'auc', **best_grid,tree_method = 'gpu_hist') #enable_categorical = True
clf_xgb.fit(X_train_all, y_train_all, verbose = False)
final_model = clf_xgb

In [37]:

y_pred_submit = final_model.predict_proba(eval_data.drop(columns=[ "ROW_ID"]))[:, 1]

## Submission

In [38]:
# y_preds = cls.predict_proba(eval_data.drop(columns=["ROW_ID"]))[:, cls.classes_ == 1].squeeze()
# Make the submission file
submission_df = pd.DataFrame({"ROW_ID": eval_data["ROW_ID"], "conversion": y_pred_submit})
submission_df["ROW_ID"] = submission_df["ROW_ID"].astype(int)
submission_df.to_csv("noche_testeo.csv", sep=",", index=False)